MySQL Query used to create the "raw" data from chemBL - note that we do filter to IC50 standard type and nM for our standard units as our only transformations prior to bringing this data into a Python Environment.  We do this to limit the size of the data ingested.  Additionally, we select on choice columns from our first table (Activities) as these columns we believed to be most important and all other columns to be irrelevant or redundant.
```
SELECT  a.activity_id,
		a.assay_id,
        a.doc_id,
        a.record_id,
        a.molregno,
        a.standard_relation,
        a.standard_value,
        a.standard_units,
        a.standard_flag,
        a.standard_type,
        a.pchembl_value,
        b.*,
        c.*,
        d.*
-- output data
INTO OUTFILE 'C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/result.tsv'
FIELDS TERMINATED BY '\t' OPTIONALLY ENCLOSED BY '"'
LINES TERMINATED BY '\n'
FROM chembl.activities a

-- Join on ligand table
inner join chembl.ligand_eff b
on a.activity_id = b.activity_id

-- Join on Compound Properties
inner join chembl.compound_properties c
on a.molregno = c.molregno

-- Join on Assays
inner join chembl.assays d
on a.assay_id = d.assay_id

where a.standard_type = 'IC50' and a.standard_units = 'nM';
```